### ResNet50으로 개와 고양이 이미지 분류 학습

In [1]:
import torchvision.transforms.v2 as v2
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [2]:
import os
from pathlib import Path

my_transform = v2.Compose([ 

             v2.RandomResizedCrop(size=(224, 224)), v2.ToTensor()])
dataset= ImageFolder('./080289/chap06/data/dogs-vs-cats', transform= my_transform)
image, label = dataset[0]
type(image), type(label)

data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

C:\Users\user\anaconda3\envs\torch-book\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
from torchvision.models import resnet50, ResNet50_Weights

model_v1 =resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

model_default =resnet50(weights=ResNet50_Weights.DEFAULT)
model_v1.fc = torch.nn.Linear(model_v1.fc.in_features, 2)  # 2개의 클래스


In [4]:
for p in model_v1.parameters():
    p.requires_grad=False

In [5]:
for p in model_v1.fc.parameters():
    p.requires_grad=True

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_v1.parameters(), lr=0.001)

In [7]:
model_v1

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
EPOCHS = 1
for n in range(EPOCHS):
    print('EPOCH {}:'.format(n + 1))
    epoch_loss = 0
    step_loss = 0
    for idx, (X_train, y_label) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model_v1(X_train)
        loss = loss_fn(outputs, y_label)
        loss.backward()
        optimizer.step()
        step_loss += loss.item()
        
        # 매 배치마다 손실값 출력
        print('  batch {} loss: {}'.format(idx + 1, step_loss))
        
    # 에포크 후 평균 손실 출력
    average_epoch_loss = step_loss / len(data_loader)
    print('  EPOCH {} average loss: {}'.format(n + 1, average_epoch_loss))

EPOCH 1:
  batch 1 loss: 0.7048102021217346
  batch 2 loss: 1.5881626605987549
  batch 3 loss: 2.3811802864074707
  batch 4 loss: 2.971734046936035
  batch 5 loss: 3.6647820472717285
  batch 6 loss: 4.672009468078613
  batch 7 loss: 5.312166631221771
  batch 8 loss: 5.735329687595367
  batch 9 loss: 6.210548460483551
  batch 10 loss: 6.6405817568302155
  batch 11 loss: 7.2090597450733185
  batch 12 loss: 7.702326953411102
  batch 13 loss: 8.115136623382568
  batch 14 loss: 8.466142505407333
  batch 15 loss: 8.821536421775818
  batch 16 loss: 9.082452952861786
  EPOCH 1 average loss: 0.5676533095538616


In [21]:
summary(model_v1)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

### Resnet model별 정확도 

In [22]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# 모델 평가
accuracy_v1 = evaluate_model(model_v1, data_loader)
accuracy_v2 = evaluate_model(model_v2, data_loader)
print("accuracy_v1", accuracy_v1)
print("accuracy_v2",accuracy_v2)

accuracy_v1 0.9103585657370518
accuracy_v2 0.9681274900398407


일반적으로: IMAGENET1K_V2가 최신 기술을 반영하고 있어 일반적으로 IMAGENET1K_V1보다 나은 성능을 발휘하지만, 특정 데이터셋에서는 IMAGENET1K_V1이 더 좋은 성능을 보일 수 있습니다. 개와 고양이와 같은 간단한 이진 분류 작업에서는 두 모델 모두 높은 정확도를 제공할 가능성이 큽니다.

### ResNet은 학습파라미터 수가 VggNet19보다 적다

In [10]:
from torchvision.models import resnet50, ResNet50_Weights
model_v2 =resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

In [11]:
model_v2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
for p in model_v2.parameters():
    p.requires_grad=False

In [13]:
for name, param in model_v2.named_parameters():
    if param.requires_grad:
        print(name, param.data)

In [14]:
from torchsummary import summary
summary(model_v2)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

In [15]:
for p in model_v2.fc.parameters():
    p.requires_grad=True

In [16]:
model_v2.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_v2.parameters(), lr=0.001)

In [18]:
model_v2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
summary(model_v2)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

In [20]:
EPOCHS = 1
for n in range(EPOCHS):
    print('EPOCH {}:'.format(n + 1))
    epoch_loss = 0
    step_loss = 0
    for idx, (X_train, y_label) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model_v2(X_train)
        loss = loss_fn(outputs, y_label)
        loss.backward()
        optimizer.step()
        step_loss += loss.item()
        
        # 매 배치마다 손실값 출력
        print('  batch {} loss: {}'.format(idx + 1, step_loss))
        
    # 에포크 후 평균 손실 출력
    average_epoch_loss = step_loss / len(data_loader)
    print('  EPOCH {} average loss: {}'.format(n + 1, average_epoch_loss))

EPOCH 1:
  batch 1 loss: 0.6470045447349548
  batch 2 loss: 1.2649938464164734
  batch 3 loss: 1.8516021370887756
  batch 4 loss: 2.4449827671051025
  batch 5 loss: 2.9900351762771606
  batch 6 loss: 3.526637554168701
  batch 7 loss: 3.957641214132309
  batch 8 loss: 4.38225457072258
  batch 9 loss: 4.760608285665512
  batch 10 loss: 5.147717386484146
  batch 11 loss: 5.536128491163254
  batch 12 loss: 5.880608648061752
  batch 13 loss: 6.248112589120865
  batch 14 loss: 6.604283183813095
  batch 15 loss: 6.868866711854935
  batch 16 loss: 7.1469686627388
  EPOCH 1 average loss: 0.446685541421175
